# Denoising Diffusion Probabilistic Models (DDPM)

Neste notebook, exploraremos a implementação de um modelo de difusão (DDPM) utilizando PyTorch. Para facilitar a compreensão dos conceitos fundamentais sem a complexidade computacional de imagens de alta resolução, aplicaremos o modelo a um dataset bidimensional.

O objetivo é aprender a distribuição $p(x)$ dos dados de forma que possamos gerar novos pontos que pertençam a essa mesma geometria, partindo de ruído aleatório. O processo se divide em:
1.  **Processo Forward (Difusão):** Adição progressiva de ruído gaussiano aos dados até que a distribuição se torne uma normal isotrópica $\mathcal{N}(0, I)$.
2.  **Processo Reverse (Desruído):** Treinamento de uma rede neural para estimar e remover o ruído em cada passo temporal, recuperando a estrutura original.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

In [ ]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device definido como: {device}")

# Semente para reprodutibilidade
torch.manual_seed(42)
np.random.seed(42)

## Preparação dos Dados

Utilizaremos uma distribuição de pontos 2D como nosso conjunto de dados $x_0$. A natureza não linear dessa distribuição é um bom teste para a capacidade generativa do modelo. Geraremos os dados sinteticamente, normalizando-os para o intervalo padrão, o que estabiliza o treinamento da rede neural.

Visualizaremos os dados originais para ter uma referência ("Ground Truth").

In [ ]:
import pandas as pd

def dino_dataset(n=8000):
    df = pd.read_csv("data/dino.tsv", sep="\t")
    df = df[df["dataset"] == "dino"]

    rng = np.random.default_rng(42)
    ix = rng.integers(0, len(df), n)
    x = df["x"].iloc[ix].tolist()
    x = np.array(x) + rng.normal(size=len(x)) * 0.15
    y = df["y"].iloc[ix].tolist()
    y = np.array(y) + rng.normal(size=len(x)) * 0.15
    x = (x/54 - 1) * 4
    y = (y/48 - 1) * 4
    X = np.stack((x, y), axis=1)
    return torch.from_numpy(X.astype(np.float32))

n_samples = 2000
batch_size = 128

X_train = dino_dataset(n_samples)
dataset = TensorDataset(X_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

plt.figure(figsize=(6, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], s=5, alpha=0.6, label='Dados Reais')
plt.title("Distribuição (x_0)")
plt.axis('equal')
plt.legend()
plt.show()

## O Processo de Difusão (Forward Process)

O processo de difusão é definido por uma cadeia de Markov que adiciona ruído aos dados em $T$ passos. A quantidade de ruído adicionada em cada passo é controlada por um *schedule* de variância $\beta_t$.

Podemos amostrar $x_t$ em qualquer passo temporal arbitrário $t$ diretamente de $x_0$ usando a propriedade de reparametrização gaussiana:

$$
q(x_t | x_0) = \mathcal{N}(x_t; \sqrt{\bar{\alpha}_t} x_0, (1 - \bar{\alpha}_t) \mathbf{I})
$$

Onde:
* $\alpha_t = 1 - \beta_t$
* $\bar{\alpha}_t = \prod_{s=1}^t \alpha_s$

Neste exemplo, usaremos $T=100$ passos, o que é suficiente para este problema simples (em imagens complexas, $T$ costuma ser 1000).

In [ ]:
# Hiperparâmetros de Difusão
n_steps = 100
beta_start = 1e-4 # 0.0001
beta_end = 0.02

# Definindo o schedule linear de beta
betas = torch.linspace(beta_start, beta_end, n_steps).to(device)

# Calculando alphas e alphas acumulados
alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# Cálculos auxiliares para a amostragem q(x_t | x_0)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

def q_sample(x_0, t, noise=None):
    """
    Amostra x_t a partir de x_0 dado um passo t.
    Forward process: x_t = sqrt(alpha_bar) * x_0 + sqrt(1 - alpha_bar) * epsilon
    """
    if noise is None:
        noise = torch.randn_like(x_0)
        
    sqrt_alpha_t = sqrt_alphas_cumprod[t].reshape(-1, 1)
    sqrt_one_minus_alpha_t = sqrt_one_minus_alphas_cumprod[t].reshape(-1, 1)
    
    return sqrt_alpha_t * x_0 + sqrt_one_minus_alpha_t * noise

## Arquitetura do Modelo (Reverse Process)

O objetivo da rede neural é prever o ruído $\epsilon$ adicionado a uma imagem ruidosa $x_t$, dado o passo de tempo $t$. Ou seja, a rede aprende $\epsilon_\theta(x_t, t)$.

Para este problema 2D, não precisamos de uma U-Net complexa com convoluções. Utilizaremos um **MLP (Multi-Layer Perceptron) condicional**. A arquitetura consiste em:

1.  **Entrada:** Coordenadas 2D $(x, y)$.
2.  **Time Embedding:** O passo de tempo $t$ é codificado usando *Sinusoidal Positional Embeddings* (similar aos Transformers) para que a rede tenha noção da "intensidade" do ruído naquele momento.
3.  **Blocos Residuais/Lineares:** Camadas densas com ativação GELU. O embedding de tempo é concatenado ou somado às features intermediárias.

A função de perda (Loss) é o erro quadrático médio (MSE) entre o ruído real adicionado e o ruído previsto pela rede:

$$
\mathcal{L} = || \epsilon - \epsilon_\theta(x_t, t) ||^2
$$

In [ ]:
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        half_dim = self.dim // 2
        embeddings = np.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

In [ ]:
class Block(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, hidden_dim), # Mantendo a dimensionalidade
            nn.GELU()
        )
        
    def forward(self, x):
        return self.net(x)

class SimpleDiffusionMLP(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128, time_dim=32):
        super().__init__()
        
        # Codificação do tempo
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.GELU()
        )
        
        # O modelo processa a concatenação do input e do tempo
        # Input layer
        self.input_layer = nn.Linear(input_dim + time_dim, hidden_dim)
        
        # Hidden layers (espécie de ResNet simples para dados tabulares)
        self.block1 = Block(hidden_dim, hidden_dim)
        self.block2 = Block(hidden_dim, hidden_dim)
        
        # Output layer (predição do ruído, mesma dimensão da entrada)
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x, t):
        # 1. Obter embedding do tempo
        t_emb = self.time_mlp(t)
        
        # 2. Concatenar x com t_emb
        # x: [batch, 2], t_emb: [batch, time_dim] -> x_input: [batch, 2+time_dim]
        x_input = torch.cat([x, t_emb], dim=1)
        
        # 3. Passar pela rede
        h = self.input_layer(x_input)
        h = h + self.block1(h) # Conexão residual simples
        h = h + self.block2(h)
        
        return self.output_layer(h)

In [ ]:
# Instanciando o modelo
model = SimpleDiffusionMLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print(model)

## Loop de Treinamento

O algoritmo de treinamento segue os passos padrão do DDPM:
1.  Amostramos um batch de dados reais $x_0$.
2.  Amostramos aleatoriamente passos de tempo $t$ para cada ponto do batch (distribuição uniforme entre $0$ e $T-1$).
3.  Geramos ruído gaussiano $\epsilon \sim \mathcal{N}(0, I)$.
4.  Criamos a versão ruidosa $x_t$ usando a função `q_sample`.
5.  O modelo tenta prever $\epsilon$ a partir de $x_t$ e $t$.
6.  Calculamos o gradiente descendente no erro MSE.

In [ ]:
n_epochs = 2000
losses = []

model.train()

for epoch in range(n_epochs):
    epoch_loss = 0
    for batch_idx, (x0_batch,) in enumerate(dataloader):
        x0_batch = x0_batch.to(device)
        batch_size_current = x0_batch.shape[0]
        
        # 1. Amostrar t uniformemente
        t = torch.randint(0, n_steps, (batch_size_current,), device=device).long()
        
        # 2. Gerar ruído aleatório
        noise = torch.randn_like(x0_batch)
        
        # 3. Gerar x_t (imagem ruidosa)
        x_t = q_sample(x0_batch, t, noise)
        
        # 4. Predição do modelo
        predicted_noise = model(x_t, t)
        
        # 5. Cálculo da Loss
        loss = nn.MSELoss()(predicted_noise, noise)
        
        # 6. Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    avg_loss = epoch_loss / len(dataloader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} | Loss: {avg_loss:.5f}")

plt.plot(losses)
plt.title("Curva de Aprendizado (Loss)")
plt.xlabel("Épocas")
plt.ylabel("MSE")
plt.show()

## Amostragem (Reverse Process)

Para gerar novos dados, começamos com ruído puro $x_T \sim \mathcal{N}(0, I)$ e iteramos para trás, de $T$ até $0$. A equação de atualização para remover o ruído é:

$$
x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left( x_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(x_t, t) \right) + \sigma_t z
$$

Onde $z \sim \mathcal{N}(0, I)$ é um ruído auxiliar que adicionamos em todos os passos (exceto no último, $t=0$) para manter a estocasticidade do processo generativo. Se não adicionarmos $z$, o processo se torna determinístico (similar ao DDIM).

Vamos gerar novos pontos e verificar se eles formam a geometria original.

In [ ]:
@torch.no_grad()
def p_sample_loop(model, shape):
    device = next(model.parameters()).device
    x = torch.randn(shape, device=device)

    for t in reversed(range(n_steps)):
        ts = torch.full((shape[0],), t, device=device, dtype=torch.long)
        eps = model(x, ts)

        a = alphas[t]
        b = betas[t]
        sqrt_oma = sqrt_one_minus_alphas_cumprod[t]

        mean = (x - b * eps / sqrt_oma) / torch.sqrt(a)
        x = mean if t == 0 else mean + torch.sqrt(b) * torch.randn_like(x)

    return x

In [ ]:
model.eval()
data = p_sample_loop(model, (1000, 2)).cpu().numpy()

plt.figure(figsize=(8,8))
plt.scatter(X_train[:,0], X_train[:,1], c='gray', alpha=0.1)
plt.scatter(data[:,0], data[:,1], c='blue', s=10)
plt.axis('equal')
plt.title("DDPM: Dados Gerados")
plt.show()

In [ ]:
@torch.no_grad()
def p_sample_trajectory(model, shape):
    device = next(model.parameters()).device
    x = torch.randn(shape, device=device)
    traj = [x.cpu().numpy()]

    for t in reversed(range(n_steps)):
        ts = torch.full((shape[0],), t, device=device, dtype=torch.long)
        eps = model(x, ts)

        a = alphas[t]
        b = betas[t]
        sqrt_oma = sqrt_one_minus_alphas_cumprod[t]

        mean = (x - b * eps / sqrt_oma) / torch.sqrt(a)
        x = mean if t == 0 else mean + torch.sqrt(b) * torch.randn_like(x)

        traj.append(x.cpu().numpy())

    return traj

In [ ]:
steps = np.linspace(0, n_steps, 6, dtype=int)
traj = p_sample_trajectory(model, (1000, 2))

fig, axs = plt.subplots(1, len(steps), figsize=(18,3))

for ax, s in zip(axs, steps):
    s = min(s, len(traj)-1)
    ax.scatter(traj[s][:,0], traj[s][:,1], s=2, c='blue')
    ax.set_title(f"t={s}")
    ax.set_xlim(-5.2,5.2)
    ax.set_ylim(-5.2,5.2)
    ax.set_aspect("equal")
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import imageio.v2 as imageio

def save_ddpm_video(traj, filename="ddpm_video.mp4", fps=20):
    frames = []

    for data in traj:
        fig, ax = plt.subplots(figsize=(4,4))
        ax.scatter(data[:,0], data[:,1], s=2)
        ax.set_xlim(-4, 4)
        ax.set_ylim(-4, 4)
        ax.set_aspect("equal")
        ax.axis("off")

        fig.canvas.draw()
        rgba = np.asarray(fig.canvas.buffer_rgba())
        rgb = rgba[..., :3]
        frames.append(rgb)

        plt.tight_layout()
        plt.close(fig)

    imageio.mimsave(filename, frames, fps=fps)
    print("Vídeo salvo em:", filename)

traj = p_sample_trajectory(model, (1000, 2))
save_ddpm_video(traj, "data/ddpm_dino.mp4", fps=20)